# MLDS/DiRP Reading Group Project 0: Neural Network Regression

For this project, you will writing a neural network using PyTorch. You will be predicting house values in California using the California Housing Prices dataset.

### Getting Started: Google Colab

On Google Colab, click connect and start running cells. All the necessary packages are pre-installed.

### Getting Started: Local

To run this project locally with conda, run the following commands:

``` bash
conda create -n reading_group python==3.10.13
conda activate reading_group
pip install torch
pip install scikit-learn
pip install juptyerlab
```

If you do not have conda, you can install the VSCode Extension `Python Environment Manager`, then it should prompt you to install conda.

### Project Tasks

1. Implement the `MLP` class
2. Define the `criterion` and `optimizer` variables
3. Experiment with different hyperparameters to get the best test loss

### Bonus Tasks

1. Add GPU support
2. Implement checkpointing
3. Implement a learning rate scheduler
4. Implement early stopping
5. Download the iris dataset using `load_iris` and train a classifier instead of a regressor

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
from sklearn.datasets import fetch_california_housing, load_iris
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
# Multi-layer perceptron
# import torch.nn as nn
class MLP(nn.Module):
    def __init__(self, input_size):
        """Define layers here"""
        super().__init__()
        self.l0=nn.Linear(input_size, 512)
        self.bn0=nn.BatchNorm1d(512)
        self.d0 = nn.Dropout(0.1)
        self.relu0 = nn.ReLU()

        self.l1=nn.Linear(512, 512)
        self.bn1=nn.BatchNorm1d(512)
        self.d1 = nn.Dropout(0.1)
        self.relu1 = nn.ReLU()

        self.l2=nn.Linear(512,512)
        self.bn2=nn.BatchNorm1d(512)
        self.d2 = nn.Dropout(0.1)
        self.relu2 = nn.ReLU()

        self.l3=nn.Linear(512, 512)
        self.bn3=nn.BatchNorm1d(512)
        self.d3 = nn.Dropout(0.1)
        self.relu3 = nn.ReLU()

        self.l4=nn.Linear(512,512)
        self.bn4=nn.BatchNorm1d(512)
        self.d4 = nn.Dropout(0.1)
        self.relu4 = nn.ReLU()

        self.l5=nn.Linear(512, 512)
        self.bn5=nn.BatchNorm1d(512)
        self.d5 = nn.Dropout(0.1)
        self.relu5 = nn.ReLU()

        self.l6=nn.Linear(512, 1)


    def forward(self, x : Tensor) -> Tensor:
        """Use layers here
        Input dimension: (batch_size, 8)
        Output dimension: (batch_size, 1)"""
        # YOUR CODE HERE

        x = self.relu0(self.d0(self.bn0(self.l0(x))))
        x = self.relu1(self.d1(self.bn1(self.l1(x))))
        x = self.relu2(self.d2(self.bn2(self.l2(x))))
        x = self.relu3(self.d3(self.bn3(self.l3(x))))
        x = self.relu4(self.d4(self.bn4(self.l4(x))))
        x = self.relu5(self.d5(self.bn5(self.l5(x))))
        x=self.l6(x)
        return x




In [ ]:
# Load the Boston housing dataset
data = fetch_california_housing()
inputs = data.data
targets = data.target

# Convert the data to tensors
inputs = torch.tensor(inputs, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.float32)

# Normalize the data
mean = inputs.mean(dim=0, keepdim=True)
std = inputs.std(dim=0, keepdim=True)
inputs = (inputs - mean) / std

mean = targets.mean(dim=0, keepdim=True)
std = targets.std(dim=0, keepdim=True)
targets = (targets - mean) / std

print("input shape", inputs.shape)
print("target shape", targets.shape)
print(data.DESCR)

dataset = TensorDataset(inputs, targets)
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1], generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)
# Create the MLP model
model = MLP(8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

input shape torch.Size([20640, 8])
target shape torch.Size([20640])
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This

In [ ]:
best_train_loss = float("inf")
best_val_loss = float("inf")
best_test_loss = float("inf")
best_epoch = 0
# Train the model
for epoch in range(100):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_dataloader:
        outputs = model(inputs)
        loss = criterion(outputs.flatten(), targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in val_dataloader:
            outputs = model(inputs)
            loss = criterion(outputs.flatten(), targets)
            val_loss += loss.item()

        # Test the model
        test_loss = 0.0
        for inputs, targets in test_dataloader:
            outputs = model(inputs)
            loss = criterion(outputs.flatten(), targets)
            test_loss += loss.item()

    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    test_loss /= len(test_dataloader)


    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch}, Train Loss: {train_loss / len(train_dataloader)}, Val Loss: {val_loss / len(val_dataloader)}, Test Loss: {test_loss / len(test_dataloader)}")


    # Checkpoint the best model
    if val_loss < best_val_loss:
        best_train_loss = train_loss
        best_val_loss = val_loss
        best_test_loss = test_loss
        best_epoch = epoch
        torch.save(model.state_dict(), "best.pt")

    # Early stopping
    if epoch - best_epoch >= 20:
        print(f"Stopping early at epoch {epoch}")
        print(
            f"Best Epoch: {best_epoch}, Train Loss: {best_train_loss:4f}, Val Loss: {best_val_loss:4f}, Test Loss: {best_test_loss:4f}"
        )
        break

    # Update the LR
    scheduler.step()

Epoch 9, Train Loss: 0.006316485108708489, Val Loss: 0.044018701638704465, Test Loss: 0.03737354977631275
Epoch 19, Train Loss: 0.0063645071108665695, Val Loss: 0.043125401676437, Test Loss: 0.03692822986178928
Epoch 29, Train Loss: 0.006230036845574012, Val Loss: 0.045073549136703396, Test Loss: 0.03701156157034415
Epoch 39, Train Loss: 0.006140389125022662, Val Loss: 0.04157633122838574, Test Loss: 0.036062419414520264
Stopping early at epoch 45
Best Epoch: 25, Train Loss: 0.407353, Val Loss: 0.366158, Test Loss: 0.328740
